In [1]:
import os
import random
import time

import numpy as np
import tensorflow as tf

import tensorflow.contrib.layers
import gym

In [2]:
import tools
reload(tools)

<module 'tools' from 'tools.pyc'>

In [4]:
class ToTheRight(gym.Env):
    def __init__(self, num):
        self.num = num
        self.state = 0.
        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.Box(0., float(num), shape=(1,))
        self.reward_range = (-1., 1.)

    def _step(self, a):
        if a == 2:
            r = 1
            self.state += 1
        elif a == 1:
            r = -0.1
        else:
            r = -1
            self.state = max(0., self.state - 1)
        return np.array([self.state]), r, self.state >= self.num, None
    
    def _reset(self):
        self.state = 0.
        return np.array([self.state])

In [35]:
env = gym.make('LunarLander-v2')

[2017-03-15 08:06:22,637] Making new env: LunarLander-v2


In [1291]:
env = ToTheRight(10)

In [6]:
def CartPoleQNetwork(state):
#     state = tf.squeeze(tf.one_hot(tf.cast(state, tf.int32), 10), -2)
    hidden1 = tf.contrib.layers.fully_connected(
        state, 32,
        activation_fn=tf.nn.elu,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
        weights_regularizer=tf.contrib.layers.l2_regularizer(0.001),
        biases_initializer=tf.zeros_initializer(),
        scope='hidden1')
    hidden2 = tf.contrib.layers.fully_connected(
        state, 32,
        activation_fn=tf.nn.elu,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
        weights_regularizer=tf.contrib.layers.l2_regularizer(0.001),
        biases_initializer=tf.zeros_initializer(),
        scope='hidden2')

    value = tf.contrib.layers.linear(hidden2, 1,
                                     weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
#                                      weights_regularizer=tf.contrib.layers.l2_regularizer(0.001),
                                     biases_initializer=tf.zeros_initializer(),
                                     scope='value')
    logits = tf.contrib.layers.linear(hidden2, env.action_space.n,
                                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
#                                       weights_regularizer=tf.contrib.layers.l2_regularizer(0.001),
                                      biases_initializer=tf.zeros_initializer(),
                                      scope='logits')

    return logits, value

In [36]:
ROLLOUT_LEN = 5
GAMMA = 0.95
TAU = 0.5

In [38]:
buf = tools.ExperienceBuffer(1 << 17)
old_s = env.reset()
done = False

for _ in range(1000):
    ss, aa, rr, ss1, gg = [], [], [], [], []
    
    g_ = 1.
    r_ = 0.
    
    while len(aa) < ROLLOUT_LEN:
        if not done:
            a = env.action_space.sample()
            s, r, done, _ = env.step(a)
        else:
            r = 0.

        g = GAMMA if not done else 0.
        r_ += g_ * r
        g_ *= g
            
        ss.append(old_s)
        aa.append(a)
        gg.append(g)
        old_s = s

    if done:
        old_s = env.reset()
        done = False
    
    ss1.append(old_s)
    buf.add(np.array([ss]), np.array([aa]), np.array(r_), np.array(ss1), np.array([gg]), 100)

In [39]:
tf.reset_default_graph()

In [40]:
state = tf.placeholder(tf.float32, shape=[None] + list(buf.ss.shape[1:]), name='state')
action = tf.placeholder(tf.int32, shape=[None] + list(buf.aa.shape[1:]), name='action')
reward = tf.placeholder(tf.float32, shape=[None], name='reward')
state1 = tf.placeholder(tf.float32, shape=[None] + list(buf.ss1.shape[1:]), name='state1')
gamma = tf.placeholder(tf.float32, shape=[None] + list(buf.gg.shape[1:]), name='gamma')

In [41]:
with tf.variable_scope('model', reuse=False):
    logits, value = CartPoleQNetwork(state)
with tf.variable_scope('model', reuse=True):
    logits1, value1 = CartPoleQNetwork(state1)

In [42]:
policy = tf.reshape(tf.multinomial(logits1, 1), [])
probs = tf.nn.softmax(logits)

In [43]:
G_unwrapped = tf.reduce_sum(tf.nn.log_softmax(logits) * tf.one_hot(action, 4), 2)
g_unwrapped = tf.cumprod(gamma, axis=1, exclusive=True)
G = tf.reduce_sum(G_unwrapped * g_unwrapped, axis=1)

In [44]:
delta_V = value[:, 0, :] - tf.reduce_prod(gamma, axis=1) * tf.stop_gradient(value1)

error = reward - delta_V - TAU * G
error_value = reward - delta_V - TAU * tf.stop_gradient(G)
error_policy = (reward - tf.stop_gradient(delta_V)) / TAU - G

In [45]:
loss = tf.reduce_mean(
    0.5 * tools.HuberLoss(error_value, 10) + tools.HuberLoss(error_policy, 10))

In [46]:
tf.summary.histogram('Monitor/TD_Error', error)
tf.summary.histogram('Monitor/Q', value)
tf.summary.histogram('Monitor/Logits', logits)
tf.summary.histogram('Monitor/Probs', probs)
tf.summary.scalar("Scalars/Q", tf.reduce_mean(value))
tf.summary.scalar("Scalars/Total_Loss", loss)
tf.summary.scalar("Scalars/Value_Loss", tf.reduce_mean(reward - delta_V))
tf.summary.scalar("Scalars/Policy_Loss", tf.reduce_mean(G))

tf.summary.scalar("Scalars/TD_Error", tf.reduce_mean(error))
tf.summary.scalar("Scalars/Delta_V", tf.reduce_mean(delta_V))
tf.summary.scalar("Scalars/G", tf.reduce_mean(G_unwrapped))

tf.summary.scalar("Scalars/Entropy", 
                  tf.reduce_mean(tf.reduce_sum(-probs * tf.nn.log_softmax(logits), axis=2)))
tf.summary.scalar("Scalars/Prob/Max", 
                  tf.reduce_mean(tf.reduce_max(probs, axis=2)))
tf.summary.scalar("Scalars/Prob/Min", 
                  tf.reduce_mean(tf.reduce_min(probs, axis=2)))

tf.contrib.layers.summarize_activations()

[]

In [47]:
# optimizer = tf.train.AdamOptimizer(0.0001)
# variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'model')
# grads_policy = optimizer.compute_gradients(tf.reduce_mean(error_policy), variables)
# grads_policy = tools.ClipGradient(grads_policy, 10.)

# grads_value = optimizer.compute_gradients(tf.reduce_mean(error_value), variables)
# grads_value = tools.ClipGradient(grads_value, 10.)

In [48]:
optimizer = tf.train.AdamOptimizer(0.0001)
variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'model')
grads = optimizer.compute_gradients(loss, variables)
grads = tools.ClipGradient(grads, 100.)

# for grad, v in grads:
#     if grad is not None:
#         tf.summary.histogram('{}/grad'.format(v.name), grad)

In [49]:
global_step = tf.Variable(0, name='global_step', trainable=False)

In [50]:
train_op = tf.group(optimizer.apply_gradients(grads, global_step))

# train_op_policy = optimizer.apply_gradients(grads_policy, global_step)
# train_op_value = optimizer.apply_gradients(grads_value, global_step)

In [51]:
tf.contrib.layers.summarize_collection(tf.GraphKeys.GLOBAL_VARIABLES)
tf.contrib.layers.summarize_activations()
summary_op = tf.summary.merge_all()

In [52]:
sess.close()
# tf.reset_default_graph()
sess = tf.InteractiveSession()

In [53]:
sess.run(tf.global_variables_initializer())

In [54]:
writer = tf.summary.FileWriter('log2/LunarLander/pcl-mul')
writer.add_graph(tf.get_default_graph())

In [55]:
def Step(ss, aa, rr, ss1, gg):
    step = sess.run(global_step)

    if step > 1 and step % 500 == 0:
        sumr, _ = sess.run([summary_op, train_op],
                           {state: ss, action: aa, reward: rr,
                            state1: ss1, gamma: gg})
        writer.add_summary(sumr, step)
    else:
        sess.run(train_op, {state: ss, action: aa, reward: rr,
                            state1: ss1, gamma: gg})
        
#     if step > 1 and step % 10000 == 0:
#         sess.run(copy_op)

In [56]:
episode_rew = 0.
episode_len = 0.
old_s = env.reset()
for i in xrange(100500100):
    ss, aa, rr, ss1, gg = [], [], [], [], []
    
    g_ = 1.
    r_ = 0.
    
    while len(aa) < ROLLOUT_LEN:
        if not done:
            a = sess.run(policy, {state1: np.expand_dims(old_s, 0)})
            s, r, done, _ = env.step(a)
            episode_rew += r
            episode_len += 1
        else:
            r = 0.

        g = GAMMA if not done else 0.
        r_ += g_ * r
        g_ *= g
            
        ss.append(old_s)
        aa.append(a)
        gg.append(g)
        old_s = s

    if done:
        writer.add_summary(tf.Summary(
            value=[tf.Summary.Value(tag='Env/Reward', simple_value=episode_rew),
                   tf.Summary.Value(tag='Env/Length', simple_value=episode_len)]),
                   sess.run(global_step))
        episode_rew = 0.
        episode_len = 0.

        old_s = env.reset()
        done = False
    
    ss1.append(old_s)
    buf.add(np.array([ss]), np.array([aa]), np.array(r_), np.array(ss1), np.array([gg]), 100)

    for _ in xrange(3):
        _, ss, aa, rr, ss1, gg, _ = buf.sample(64)
        Step(ss, aa, rr, ss1, gg)

KeyboardInterrupt: 

In [1325]:
buf.inserted, buf.buffer_size

(26719, 262144)